# Базовый Python
# Проект: Музыка больших городов

## 1. Постановка задачи

Сравнение Москвы и Петербурга окружено мифами:
 - Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 - Петербург — город своеобразной культуры, непохожий на Москву.

Некоторые мифы отражают действительность. Другие — пустые стереотипы. Бизнес должен отличать первые от вторых, чтобы принимать рациональные решения. На реальных данных Яндекс Музыки вы проверите гипотезы и сравните поведение пользователей двух столиц.

### Гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. Утром в понедельник в Москве преобладают одни жанры музыки, а в Петербурге — другие. Это верно и для вечера пятницы.
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

### Данные 

Данные хранятся в файле `yandex_music_project.csv`. Скачать датасет можно здесь.

**Описание колонок:**
 - `userID` — идентификатор пользователя
 - `Track` — название трека
 - `artist` — имя исполнителя
 - `genre` — название жанра
 - `City` — город пользователя
 - `time` — время начала прослушивания
 - `Day` — день недели

4. Анализ данных
5. Оформление результатов

## 2. Получение данных

Прочтём данные и проведём ознакомительный анализ.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/yandex_music_project.csv')

df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


**Выводы:**

1. В таблице содержится информация о пользователях и треках, которые они прослушивали.
2. В таблице семь столбцов типа — `object`:
    - `userID` — идентификатор пользователя
    - `Track` — название трека
    - `artist` — имя исполнителя
    - `genre` — название жанра
    - `City` — город пользователя
    - `time` — время начала прослушивания
    - `Day` — день недели
3. Количество значений в столбцах различается - в данных есть пропущенные значения.
4. В наименовании атрибутов таблицы наблюдаются три типа нарушения стиля. Необходимо их исправить:
    - Строчные буквы сочетаются с прописными.
    - Встречаются пробелы.
    - Данные записаны в стиле camelCase, необходимо перевести в snake_case
5. Данных достаточно, для проведения проверки гипотез.

## 3. Предобработка данных

### 3.1. Стилистика наименования атрибутов таблицы

Исправим стиль наименований атрибутов таблицы.

In [4]:
print('Список наименований столбцов таблицы:')
print('    - ДО преобразования:   ', list(df.columns))

df = df.rename(
    columns={
        '  userID': 'user_id',
        'Track': 'track',
        '  City  ': 'city',
        'Day': 'day'
    }
)

print('    - ПОСЛЕ преобразования:', list(df.columns))

Список наименований столбцов таблицы:
    - ДО преобразования:    ['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day']
    - ПОСЛЕ преобразования: ['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day']


### 3.2. Пропуски значениях таблицы

Посмотрим сколько пустых значений содержится в столбцах таблицы.

Не все пропущенные значения влияют на исследование. Например, в атрибутах `track` и `artist` пропуски не важны для нашей работы. Достаточно заменить их явными обозначениями. Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге.

На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Придётся:
- заполнить и эти пропуски явными обозначениями
- оценить, насколько они повредят расчётам

In [7]:
df \
    .isna()\
    .sum() \
    .reset_index() \
    .rename(columns={'index': 'column_name', 0: 'is_na_cnt'}) \
    .sort_values(by='is_na_cnt', ascending=False) \
    .reset_index(drop=True)

,column_name,is_na_cnt
0,artist,7203
1,track,1231
2,genre,1198
3,user_id,0
4,city,0
5,time,0
6,day,0


Заменим пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`.

In [8]:
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
for curr_column in ['track', 'artist', 'genre']:
    df[curr_column] = df[curr_column].fillna('unknown')

Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитайте пропущенные значения.

In [9]:
df \
    .isna()\
    .sum() \
    .reset_index() \
    .rename(columns={'index': 'column_name', 0: 'is_na_cnt'}) \
    .sort_values(by='is_na_cnt', ascending=False) \
    .reset_index(drop=True)

,column_name,is_na_cnt
0,user_id,0
1,track,0
2,artist,0
3,genre,0
4,city,0
5,time,0
6,day,0


### 3.3. Дубликаты

**Явные дубли**

Оценим объем явных дубликатов и скорректируем их.

In [7]:
print('Количество дубликатов:')

print(f'    - ДО корректировки: {df.duplicated().sum()} шт.')

df = df.drop_duplicates().reset_index(drop=True)

print(f'    - ПОСЛЕ корректировки: {df.duplicated().sum()} шт.')

Количество дубликатов:
    - ДО корректировки: 0 шт.
    - ПОСЛЕ корректировки: 0 шт.


**Неявные дубли**

Теперь избавимся от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному.

Выведем на экран список уникальных названий жанров, отсортированный в алфавитном порядке.

In [11]:
print('Список уникальных названий жанров:', df['genre'].sort_values().unique(), sep='\n')

Список уникальных названий жанров:
['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitness' 'flamenco' 'folk' 

Удалось обнаружить неявные дубликаты названия ***hiphop*** - ***hip***, ***hop***, ***hip-hop***. Скорректируем их.

In [12]:
df['genre'] = df['genre'].replace(['hip', 'hop', 'hip-hop'], 'hiphop')

In [13]:
print('Список уникальных названий жанров после преобразования:', df['genre'].sort_values().unique(), sep='\n')

Список уникальных названий жанров после преобразования:
['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'alternativepunk' 'ambient' 'americana' 'animated' 'anime' 'arabesk'
 'arabic' 'arena' 'argentinetango' 'art' 'audiobook' 'author' 'avantgarde'
 'axé' 'baile' 'balkan' 'beats' 'bigroom' 'black' 'bluegrass' 'blues'
 'bollywood' 'bossa' 'brazilian' 'breakbeat' 'breaks' 'broadway'
 'cantautori' 'cantopop' 'canzone' 'caribbean' 'caucasian' 'celtic'
 'chamber' 'chanson' 'children' 'chill' 'chinese' 'choral' 'christian'
 'christmas' 'classical' 'classicmetal' 'club' 'colombian' 'comedy'
 'conjazz' 'contemporary' 'country' 'cuban' 'dance' 'dancehall' 'dancepop'
 'dark' 'death' 'deep' 'deutschrock' 'deutschspr' 'dirty' 'disco' 'dnb'
 'documentary' 'downbeat' 'downtempo' 'drum' 'dub' 'dubstep' 'eastern'
 'easy' 'electronic' 'electropop' 'emo' 'entehno' 'epicmetal' 'estrada'
 'ethnic' 'eurofolk' 'european' 'experimental' 'extrememetal' 'fado'
 'fairytail' 'film' 'fitnes

**Выводы**

В процессе предобработки данных удалось устранить следующие проблемы:
- Нарушения в стиле заголовков – для упрощения работы с таблицей,
- пропущенные значения,
- Явные и неявные дубликаты.

## 4. Проверка гипотез

**Гипотеза №1. Сравнение поведения пользователей двух столиц.**

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. 


1. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:
    - Разделим пользователей Москвы и Санкт-Петербурга
    - Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.
2. Оценим активность пользователей в каждом городе. Сгруппируем данные по городу и посчитаем прослушивания в каждой группе.

Видим, что Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Что касается дней недель - в среднем пользователи из обоих городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

In [8]:
print('Кол-во прослушиваний в каждом городе:', df.groupby('city')['city'].count(), sep='\n')

Кол-во прослушиваний в каждом городе:
city
Moscow              42741
Saint-Petersburg    18512
Name: city, dtype: int64


In [14]:
print('Кол-во прослушиваний в каждый из трёх дней недели:', df.groupby('day')['day'].count(), sep='\n')

Кол-во прослушиваний в каждый из трёх дней недели:
day
Friday       21840
Monday       21354
Wednesday    18059
Name: day, dtype: int64


Создадим функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города. 

In [16]:
def number_tracks(day, city):
    """
    Подсчитывает количество прослушиваний треков для указанного дня и города.
    
    Параметры:
    ----------
    day : str
        День недели для фильтрации (например, 'Monday', 'Friday')
    city : str  
        Название города для фильтрации (например, 'Moscow', 'Saint-Petersburg')
        
    Возвращает:
    -----------
    int
        Количество прослушиваний (записей) для указанного дня и города
        
    Принцип работы:
    ---------------
    1. Фильтрует DataFrame по дню недели
    2. Из отфильтрованных данных выбирает записи для указанного города  
    3. Подсчитывает количество записей в столбце 'user_id'
    4. Возвращает полученное число
    """
    
    # Шаг 1: Фильтрация по дню недели
    # Получаем строки, где значение в столбце 'day' равно параметру day
    track_list = df[df['day'] == day]
    
    # Шаг 2: Фильтрация по городу
    # Из уже отфильтрованных данных выбираем строки для нужного города
    track_list = track_list[track_list['city'] == city]
    
    # Шаг 3: Подсчет количества записей
    # Считаем количество непустых значений в столбце 'user_id'
    track_list_count = track_list['user_id'].count()
    
    # Шаг 4: Возврат результата
    return track_list_count

In [25]:
print(f'Кол-во прослушиваний в Москве по понедельникам: {number_tracks('Monday', 'Moscow')} шт.')
print(f'Кол-во прослушиваний в Санкт-Петербурге по понедельникам: {number_tracks('Monday', 'Saint-Petersburg')} шт.')

Кол-во прослушиваний в Москве по понедельникам: 15740 шт.
Кол-во прослушиваний в Санкт-Петербурге по понедельникам: 5614 шт.


In [26]:
print(f'Кол-во прослушиваний в Москве по средам: {number_tracks('Wednesday', 'Moscow')} шт.')
print(f'Кол-во прослушиваний в Санкт-Петербурге по средам: {number_tracks('Wednesday', 'Saint-Petersburg')} шт.')

Кол-во прослушиваний в Москве по средам: 11056 шт.
Кол-во прослушиваний в Санкт-Петербурге по средам: 7003 шт.


In [27]:
print(f'Кол-во прослушиваний в Москве по пятницам: {number_tracks('Friday', 'Moscow')} шт.')
print(f'Кол-во прослушиваний в Санкт-Петербурге по пятницам: {number_tracks('Friday', 'Saint-Petersburg')} шт.')

Кол-во прослушиваний в Москве по пятницам: 15945 шт.
Кол-во прослушиваний в Санкт-Петербурге по пятницам: 5895 шт.


Проделаем тоже самое при помощи `Pandas`:

In [32]:
pd.pivot_table(
    df, 
    index='city',
    columns='day',
    aggfunc='size'
)

day,Friday,Monday,Wednesday
city,,,
Moscow,15945,15740,11056
Saint-Petersburg,5895,5614,7003


**Выводы**

Гипотеза №1 подтверждена. Поведение пользователей между городами отличается:
- Пик прослушиваний в  Москве приходится на понедельник и пятницу. В среду наблюдается спад.
- В Петербурге, наоборот, большее кол-во прослушиваний в средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

### Гипотеза №2. Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве и в Петербурге преобладают разные жанры. Такое же предпложение допускается по поводу вечера пятницы.

In [37]:
moscow_general = df[df['city'] == 'Moscow']
spb_general = df[df['city'] == 'Saint-Petersburg']

Создадим функцию `genre_weekday()`, которая вернет информацию о топ-10 жанрах треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени. Функция  с четырьмя параметрами:
* таблица (датафрейм) с данными
* день недели
* начальная временная метка в формате 'hh:mm' 
* последняя временная метка в формате 'hh:mm'

In [43]:
def genre_weekday(table, day, time1, time2):
    """
    Анализирует популярность музыкальных жанров в указанный день недели 
    в заданном временном интервале.
    
    Функция фильтрует данные по дню недели и временному диапазону, 
    затем определяет топ-10 наиболее популярных жанров на основе 
    количества прослушиваний.
    
    Параметры:
    ----------
    table : pandas.DataFrame
        Исходный датафрейм с данными о прослушиваниях.
        Обязательные столбцы: 'day', 'time', 'genre'
    day : str
        День недели для фильтрации (например: 'Monday', 'Tuesday')
    time1 : str или datetime
        Начальное время интервала (не включается в выборку)
        Формат зависит от типа данных в столбце 'time'
    time2 : str или datetime  
        Конечное время интервала (не включается в выборку)
        
    Возвращает:
    -----------
    pandas.Series
        Series с топ-10 жанров, отсортированный по убыванию популярности.
        Индекс - названия жанров, значения - количество прослушиваний.
        
    Пример использования:
    --------------------
    >>> # Популярные жанры в понедельник с 07:00 до 11:00
    >>> top_genres = genre_weekday(df, 'Monday', '07:00', '11:00')
    >>> print(top_genres.head(3))
    genre
    pop       1247
    rock       986
    hip-hop    743
    Name: genre, dtype: int64
    
    Алгоритм работы:
    ----------------
    1. Фильтрация данных по трем условиям:
       - day == указанный день
       - time > время начала интервала  
       - time < время окончания интервала
    2. Группировка отфильтрованных данных по жанрам
    3. Подсчет количества записей для каждого жанра
    4. Сортировка по убыванию популярности
    5. Возврат топ-10 результатов
    """
    
    # Шаг 1: Фильтрация данных по дню недели и временному интервалу
    # Используется логическая индексация с тремя условиями
    genre_df = table[
        (table['day'] == day)           # точное совпадение дня недели
        & (table['time'] > time1)       # время больше начала интервала
        & (table['time'] < time2)       # время меньше конца интервала  
    ]
    
    # Шаг 2: Группировка по жанрам и подсчет количества записей
    # groupby('genre') - группирует строки по значениям столбца 'genre'
    # ['genre'].count() - подсчитывает количество непустых значений в каждой группе
    genre_df_count = genre_df.groupby('genre')['genre'].count()
    
    # Шаг 3: Сортировка по убыванию популярности
    # ascending=False означает сортировку от большего к меньшему
    genre_df_sorted = genre_df_count.sort_values(ascending=False)
    
    # Шаг 4: Возврат топ-10 наиболее популярных жанров
    # head(10) возвращает первые 10 элементов отсортированного Series
    return genre_df_sorted.head(10)

Cравните результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [52]:
print('1. Вечер пятницы в Москве:')
display(genre_weekday(moscow_general, 'Monday', '07:00', '11:00'))

print('2. Вечер пятницы в Петербурге:')
display(genre_weekday(spb_general, 'Monday', '07:00', '11:00'))

1. Вечер пятницы в Москве:


genre
pop            781
dance          549
electronic     480
rock           474
hip            281
ruspop         186
world          181
rusrap         175
alternative    164
classical      157
Name: genre, dtype: int64

2. Вечер пятницы в Петербурге:


genre
pop            218
dance          182
rock           162
electronic     147
hip             79
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [51]:
print('1. Вечер пятницы в Москве:')
display(genre_weekday(moscow_general, 'Friday', '17:00', '23:00'))

print('2. Вечер пятницы в Петербурге:')
display(genre_weekday(spb_general, 'Friday', '17:00', '23:00'))

1. Вечер пятницы в Москве:


genre
pop            713
rock           517
dance          495
electronic     482
hip            267
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

2. Вечер пятницы в Петербурге:


genre
pop            256
rock           216
electronic     216
dance          210
hip             94
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Гипотеза №3. Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируйте таблицу `moscow_general` по жанру и посчитайте прослушивания треков каждого жанра методом `count()`. Затем отсортируйте результат в порядке убывания и сохраните его в таблице `moscow_genres`.

In [38]:
# одной строкой: группировка таблицы moscow_general по столбцу 'genre', 
# подсчёт числа значений 'genre' в этой группировке методом count(), 
# сортировка получившегося Series в порядке убывания и сохранение в moscow_genres
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)

Выведите на экран первые десять строк `moscow_genres`:

In [39]:
# просмотр первых 10 строк moscow_genres
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<h2> Комментарий ревьюера</h2>
    
Можно и это вывести как таблицу с помощью метода **to_frame**:

</div>

In [40]:
# КОД РЕВЬЮЕРА

moscow_genres.to_frame().head(10)

,genre
genre,
pop,5892
dance,4435
rock,3965
electronic,3786
hiphop,2096
classical,1616
world,1432
alternative,1379
ruspop,1372


Теперь повторите то же и для Петербурга.

Сгруппируйте таблицу `spb_general` по жанру. Посчитайте прослушивания треков каждого жанра. Результат отсортируйте в порядке убывания и сохраните в таблице `spb_genres`:


In [41]:
# одной строкой: группировка таблицы spb_general по столбцу 'genre', 
# подсчёт числа значений 'genre' в этой группировке методом count(), 
# сортировка получившегося Series в порядке убывания и сохранение в spb_genres
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)

Выведите на экран первые десять строк `spb_genres`:

In [42]:
# просмотр первых 10 строк spb_genres
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Вы проверили три гипотезы и установили:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

**На практике исследования содержат проверки статистических гипотез.**
Из данных одного сервиса не всегда можно сделать вывод о всех жителях города.
Проверки статистических гипотез покажут, насколько они достоверны, исходя из имеющихся данных. 
С методами проверок гипотез вы ещё познакомитесь в следующих темах.

<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<h2> Комментарий ревьюера</h2>
    
Общий вывод это очень важно. Зачастую подобные задачи заканчиваются составлением презентаций или отчетов для заказчика. Поэтому все результаты нужно описывать в общем выводе. В идеале надо подкреплять его полученными ранее значениями. Так что рекомендую  во всех следующих проектах не забывать о нем :) 

</div>